In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor as gb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing 
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [51]:
from sklearn import preprocessing
train = pd.read_csv("drive/My Drive/Colab Notebooks/CMPSC 448/xgboost/train_data.csv")

train = train.drop("key", axis = 1)

X, y = train.iloc[:,:-1],train.iloc[:,-1]

for i in range(len(y)):
    if y[i] == "Class_1":
        y[i] = 0
    if y[i] == "Class_2":
        y[i] = 1
    if y[i] == "Class_3":
        y[i] = 2
    if y[i] == "Class_4":
        y[i] = 3
    if y[i] == "Class_5":
        y[i] = 4
    if y[i] == "Class_6":
        y[i] = 5
    if y[i] == "Class_7":
        y[i] = 6
    if y[i] == "Class_8":
        y[i] = 7
    if y[i] == "Class_9":
        y[i] = 8
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
eval_set = [(X_test, y_test)]
y_test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarni

60561    1
55802    1
41935    3
45242    2
55996    2
        ..
44793    1
50508    2
37421    7
46530    7
8304     1
Name: category, Length: 18564, dtype: object

In [61]:
from sklearn.calibration import CalibratedClassifierCV

xgb_clf = xgb.XGBClassifier(objective = "multi:softmax", max_depth=8, nfold = 5,nrounds = 10,n_estimators=525,min_child_weight=3, sample_type='weighted',colsample_bytree=0.7,subsample=0.7,eta=0.1,seed=4)
calibrated_xgb = CalibratedClassifierCV(xgb, cv = 5, method = 'isotonic')
xgb_fit = xgb_clf.fit(X_train, y_train, eval_set = eval_set, eval_metric = 'mlogloss', early_stopping_rounds =15)


[0]	validation_0-mlogloss:1.96859
Will train until validation_0-mlogloss hasn't improved in 15 rounds.
[1]	validation_0-mlogloss:1.81259
[2]	validation_0-mlogloss:1.68217
[3]	validation_0-mlogloss:1.56873
[4]	validation_0-mlogloss:1.47625
[5]	validation_0-mlogloss:1.39511
[6]	validation_0-mlogloss:1.32428
[7]	validation_0-mlogloss:1.26386
[8]	validation_0-mlogloss:1.20851
[9]	validation_0-mlogloss:1.1598
[10]	validation_0-mlogloss:1.11482
[11]	validation_0-mlogloss:1.07233
[12]	validation_0-mlogloss:1.03791
[13]	validation_0-mlogloss:1.00543
[14]	validation_0-mlogloss:0.974287
[15]	validation_0-mlogloss:0.945592
[16]	validation_0-mlogloss:0.918899
[17]	validation_0-mlogloss:0.895095
[18]	validation_0-mlogloss:0.872729
[19]	validation_0-mlogloss:0.852073
[20]	validation_0-mlogloss:0.832629
[21]	validation_0-mlogloss:0.814783
[22]	validation_0-mlogloss:0.798469
[23]	validation_0-mlogloss:0.783005
[24]	validation_0-mlogloss:0.768724
[25]	validation_0-mlogloss:0.756076
[26]	validation_0-ml

In [64]:
from sklearn.metrics import log_loss
preds_y = xgb_fit.predict_proba(X_test)
log_loss(y_test, preds_y)

ValueError: ignored

In [50]:
selection = SelectFromModel(xgb_fit, threshold = thresholds[0], prefit = True)
select_X_train = selection.transform(X_train)
select_X_test = selection.transform(X_test)
selection_model = xgb.XGBClassifier(objective = "multi:softmax", max_depth=250, tree_method = 'hist', grow_policy='lossguide', normalize_type = 'forest', csv_weights=10, num_round = 1200, nfold = 5,nrounds = 10,n_estimators=15000,min_child_weight=35, sample_type='weighted',colsample_bytree=0.7,subsample=0.8,eta=0.0001,seed=4)
eval_set = [(select_X_test, y_test)]
selection_model.fit(select_X_train, y_train, eval_set = eval_set, eval_metric="mlogloss", early_stopping_rounds=15)

[0]	validation_0-mlogloss:1.98602
Will train until validation_0-mlogloss hasn't improved in 15 rounds.
[1]	validation_0-mlogloss:1.82317
[2]	validation_0-mlogloss:1.69254
[3]	validation_0-mlogloss:1.5876
[4]	validation_0-mlogloss:1.49544
[5]	validation_0-mlogloss:1.41734
[6]	validation_0-mlogloss:1.3443
[7]	validation_0-mlogloss:1.28422
[8]	validation_0-mlogloss:1.22993
[9]	validation_0-mlogloss:1.17837
[10]	validation_0-mlogloss:1.13369
[11]	validation_0-mlogloss:1.09253
[12]	validation_0-mlogloss:1.05558
[13]	validation_0-mlogloss:1.02033
[14]	validation_0-mlogloss:0.990855
[15]	validation_0-mlogloss:0.962167
[16]	validation_0-mlogloss:0.934676
[17]	validation_0-mlogloss:0.910086
[18]	validation_0-mlogloss:0.887145
[19]	validation_0-mlogloss:0.866125
[20]	validation_0-mlogloss:0.847083
[21]	validation_0-mlogloss:0.830098
[22]	validation_0-mlogloss:0.814776
[23]	validation_0-mlogloss:0.799747
[24]	validation_0-mlogloss:0.784916
[25]	validation_0-mlogloss:0.772186
[26]	validation_0-mlo

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, csv_weights=10,
              eta=0.0001, gamma=0, grow_policy='lossguide', learning_rate=0.1,
              max_delta_step=0, max_depth=250, min_child_weight=35,
              missing=None, n_estimators=15000, n_jobs=1, nfold=5,
              normalize_type='forest', nrounds=10, nthread=None, num_round=1200,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, sample_type='weighted', scale_pos_weight=1, seed=4,
              silent=None, subsample=0.8, ...)

ValueError: ignored

In [65]:
test = pd.read_csv("drive/My Drive/Colab Notebooks/CMPSC 448/xgboost/test_data.csv")
test = test.drop("key", axis = 1)
test = scaler.transform(test)
preds = xgb_fit.predict_proba(test)
preds

array([[1.40603646e-04, 1.41191930e-02, 2.93561779e-02, ...,
        9.38784659e-01, 7.62482581e-04, 5.68940741e-05],
       [3.20189429e-04, 5.03201227e-05, 9.70531473e-05, ...,
        1.16816731e-02, 2.40472684e-04, 1.13915130e-02],
       [1.65275924e-05, 2.58956826e-03, 9.97115016e-01, ...,
        1.59768955e-04, 8.84291967e-06, 3.98922020e-06],
       ...,
       [1.70081714e-03, 1.11803986e-01, 8.54749680e-01, ...,
        9.33976844e-03, 3.55991675e-03, 3.58170597e-04],
       [2.39452999e-03, 3.34197975e-04, 1.86379693e-04, ...,
        2.94993538e-03, 3.35492427e-03, 1.11778395e-03],
       [1.34182497e-04, 2.03136474e-01, 7.77165294e-01, ...,
        2.89627956e-03, 1.71542197e-04, 3.35980731e-05]], dtype=float32)

In [66]:
samp_sub = pd.read_csv("drive/My Drive/Colab Notebooks/CMPSC 448/xgboost/sample_submission.csv")
output = pd.DataFrame(preds,columns=["group_0"+str(i) for i in range(1,10)])
output.insert(0, 'key', samp_sub["key"])
output

,key,group_01,group_02,group_03,group_04,group_05,group_06,group_07,group_08,group_09
0,45451,0.000141,0.014119,0.029356,0.000381,0.000008,0.016390,0.938785,0.000762,0.000057
1,63656,0.000320,0.000050,0.000097,0.000489,0.000005,0.975724,0.011682,0.000240,0.011392
2,74635,0.000017,0.002590,0.997115,0.000099,0.000001,0.000006,0.000160,0.000009,0.000004
3,89761,0.013106,0.802355,0.080980,0.025862,0.000384,0.014910,0.019465,0.010332,0.032607
4,3559,0.002081,0.903919,0.065568,0.007215,0.000535,0.001293,0.015494,0.003740,0.000156
...,...,...,...,...,...,...,...,...,...,...
144363,28524,0.000055,0.209779,0.409168,0.364714,0.000122,0.000411,0.015454,0.000181,0.000115
144364,135857,0.003317,0.795596,0.181822,0.001416,0.000026,0.001051,0.015007,0.001322,0.000443
144365,76667,0.001701,0.111804,0.854750,0.006876,0.000111,0.011501,0.009340,0.003560,0.000358
144366,93453,0.002395,0.000334,0.000186,0.006380,0.000011,0.983271,0.002950,0.003355,0.001118


In [0]:
output.to_csv("submission4.csv", index = False)
!cp submission4.csv "drive/My Drive/Colab Notebooks/CMPSC 448/xgboost/"